In [1]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [2]:
query_1 = """SELECT * FROM sfdc.accounts sfdc_accounts
LEFT JOIN (SELECT MAX(dt) FROM sfdc.cases) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' 
"""


In [ ]:
query_2 = """SELECT *
FROM (
  SELECT u.name, b.customer_tier_c, b.name, m.account, b.x18_digit_account_id_c, s.id, m.platform, m.mobile_os, m.num_requests, Row_number() OVER(
    PARTITION BY s.id
  ) row_
  FROM wbr.map_requests_by_account m
  INNER JOIN (
    SELECT DISTINCT id
    FROM mapbox_customer_data.styles 
    WHERE cast(dt as DATE) >= CURRENT_DATE - INTERVAL '14' DAY
      AND sources LIKE '%mapbox-streets-v7%'
  ) s
  ON m.service_metadata_version = s.id
  LEFT JOIN (
    SELECT customer_tier_c, csm_c, name, mapbox_username_c, x18_digit_account_id_c
    FROM sfdc.accounts
    WHERE cast(dt as DATE) = CURRENT_DATE - INTERVAL '1' DAY
  ) b
  ON m.account = b.mapbox_username_c
  LEFT JOIN (
    SELECT name, id
    FROM sfdc.users
    WHERE cast(dt as DATE) = CURRENT_DATE - INTERVAL '1' DAY
  ) u
  ON b.csm_c = u.id
  WHERE cast(m.dt as DATE) >= CURRENT_DATE - INTERVAL '14' DAY
    AND m.service_metadata = 'custom'
    AND m.service = 'styles'
    AND b.customer_tier_c IN (
      'Tier 0',
      'Tier 1',
      'Tier 2',
      'Tier 3',
      'Tier 4'
    )
)
WHERE row_ = 1
AND m.service_metadata = 'custom'
ORDER BY 1, 4
LIMIT 5000
"""

In [3]:
query_3 = """SELECT * FROM (SELECT *\nFROM (\n  with reg_users as (\n  \n  \tselect * \n  \tfrom (\n  \tselect \n  \t        a.*\n  \t        , b.*\n  \t        , c.*\n  \t        , d.*\n  \tfrom \n  \t(\n  \tselect \n  \t        distinct \n  \t        anonymous_id\n  \t        , user_id\n  \tfrom mapbox_customer_data.segment_identifies\n  \twhere dt >= \'2018-07-01\'\n  \tand anonymous_id is not null\n  \tand user_id is not null\n  \t) a\n  \n  \tleft join \n  \n  \t(\n  \tselect \n  \t        id \n  \t        , email\n  \t        , created\n  \tfrom mapbox_customer_data.accounts\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \t) b\n  \t        on a.user_id = b.id\n  \n  \tleft join \n  \n  \t(\n  \t    \n  \t        select        \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(timestamp) as min_exposure\n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        group by 1,2\n  \n  \t) c \n  \t        on a.anonymous_id = c.anon_id_ad\n  \t        \n  \tleft join \n  \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) d\n  \t        on a.anonymous_id = d.anon_id_event\n  \n  \t) \n  \twhere context_campaign_name is not null\n  \n  ), \n  \n  non_reg_users as (\n  \n  \tselect \n  \t        context_campaign_name\n  \t        , min_exposure\n  \t        , event\n  \t        , original_timestamp as event_timestamp\n  \t        , context_traits_email as event_email\n  \tfrom (\n  \tselect a.*\n  \t        , b.*\n  \tfrom \n  \t(\n  \t        select \n  \t                anonymous_id as anon_id_ad\n  \t                , context_campaign_name\n  \t                , min(original_timestamp) as min_exposure\n  \t        from (       \n  \t        select        \n  \t                context_campaign_name\n  \t                , anonymous_id\n  \t                , original_timestamp \n  \t        from mapbox_customer_data.segment_pages\n  \t        where dt >= \'2018-07-01\'\n  \t        and context_campaign_name is not null\n  \t        )\n  \t        group by 1,2\n  \t) a\n  \n  \tleft join \n  \t(\n  \t        select \n  \t                distinct\n  \t                anonymous_id as anon_id_event\n  \t                , original_timestamp\n  \t                , event\n  \t                , context_traits_email\n  \t        from mapbox_customer_data.segment_tracks\n  \t        where dt >= \'2018-07-01\'\n  \t        and event like \'submitted_%form\'\n  \t        and context_traits_email is not null\n  \t) b\n  \t        on a.anon_id_ad = b.anon_id_event\n  \n  \t)\n  \twhere anon_id_event is not null\n  \tand to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \n  ), \n  \n  mql_flag as (\n  \n  \tselect \n  \t        email\n  \t        , created_date\n  \t        , last_mql_date_c\n  \t        , mql_flag\n  \tfrom (\n  \n  \tselect \n  \t        email\n  \t        , min(created_date) created_date\n  \t        , max(last_mql_date_c) last_mql_date_c\n  \t        , case when max(last_mql_date_c) is not null then 1 else 0 end as mql_flag\n  \t        , sum(case when is_deleted = true then 1 else 0 end) as is_deleted\n  \tfrom sales.salesforce_leads\n  \twhere cast(dt as DATE) = CURRENT_DATE - INTERVAL \'1\' DAY \n  \tgroup by 1\n  \t)\n  \twhere mql_flag = 1\n  \tand is_deleted = 0\n  \n  ),\n  \n  cleaned_list as (\n  \n  \n  \tselect \n  \t\tdistinct \n  \t\t\t*\n  \tfrom (\n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , \'created_an_account\' as event\n  \t\t        , created as event_timestamp\n  \t\t        , email as event_email\n  \t\tfrom reg_users        \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(created)\n  \t\tand cast(min_exposure as DATE) >= cast(created as DATE) - INTERVAL \'28\' DAY\n  \n  \n  \t\tunion all\n  \n  \n  \t\tselect \n  \t\t        context_campaign_name\n  \t\t        , min_exposure\n  \t\t        , event\n  \t\t        , original_timestamp as event_timestamp\n  \t\t        , context_traits_email as event_email\n  \t\tfrom reg_users      \n  \t\twhere to_unixtime(min_exposure) <= to_unixtime(original_timestamp)\n  \t\tand cast(min_exposure as DATE) >= cast(original_timestamp as DATE) - INTERVAL \'28\' DAY\n  \n  \t\tunion all\n  \n  \t\tselect * \n  \t\tfrom non_reg_users\n  \t)\n  \n  )\n  \n  \n  \n  select \n  \ta.*\n  \t, b.*\n  from cleaned_list a\n  left join mql_flag b \n  \ton a.event_email = b.email\n) "custom_sql_query"\nLIMIT 0) T LIMIT 0
"""


In [2]:
query_3 = """WITH opportunity_to_name AS (
                SELECT  -- make sure there is only one name per id
                id AS account_id, name AS account_name FROM sfdc.accounts sfdc_accounts
                WHERE dt = '{run_date}' GROUP BY id, name) SELECT * FROM opportunity_to_name
                """

In [3]:
formatter = column_parser.Parser(query_3)
formatted = formatter.format_query(query_3)
query_list = formatted.split('\n')
query_list

['WITH opportunity_to_name AS',
 '  (SELECT id AS account_id,',
 '          name AS account_name',
 '   FROM sfdc.accounts sfdc_accounts',
 "   WHERE dt = '{run_date}'",
 '   GROUP BY id,',
 '            name)',
 'SELECT *',
 'FROM opportunity_to_name']

In [4]:
def delevel_query(query_list):
    line_level = []
    sub_query = []
    pos_delete, pos_where = len(query_list), len(query_list)
    
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete = i
        if line.startswith('WHERE'):
            pos_where = i 
        if line.startswith('FROM'):
            pos_from = i-1
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join = i
  
    for line in query_list[:pos_from+2]:
        line_level.append((line, "level_1"))
        
    for line in query_list[pos_from+1:pos_where]:
        if line.startswith(' ') or line.startswith('FROM'):
            sub_query.append(line[3:])
        else:
            line_level.append((line, "level_1"))

    for line in query_list[pos_where:pos_delete]:    
        line_level.append((line, "level_1"))

    return line_level, sub_query[1:]

In [5]:
def parse_sub_query(sub_query_list):
    sub_query = "\n".join(sub_query_list)
    sub_query = sub_query.lstrip('\n').lstrip(' ')
    
    formatter = column_parser.Parser(sub_query)
    formatted = formatter.format_query(sub_query)
    sub_query_list = formatted.split('\n')

    query = "\n".join(sub_query_list).lstrip('\n').lstrip(' ').lstrip('\(')
    query_list = query.split('\n')
    
    return query_list

In [6]:
def has_child(sub_query_list):
    query_list = parse_sub_query(sub_query_list)
    query_list = delevel_query(query_list)[1]
    return query_list, query_list != []

In [7]:
# delevel_query(query_list)[0]

In [8]:
formatter = column_parser.Parser(query_3)
formatted = formatter.format_query(query_3)
query_list = formatted.split('\n')
query_list, has_subquery = has_child(query_list)

In [9]:
query_list

[]

In [10]:
has_subquery

False

In [11]:
# sub_query_2_list = query_list
# sub_query_2_list

In [12]:
query_list, has_subquery = has_child(query_list)

UnboundLocalError: local variable 'pos_from' referenced before assignment

In [14]:
query_list

['WITH reg_users AS',
 '  (SELECT *',
 '   FROM',
 '     (SELECT a.*,',
 '             b.*,',
 '             c.*,',
 '             d.*',
 '      FROM',
 '        (SELECT DISTINCT anonymous_id,',
 '                         user_id',
 '         FROM mapbox_customer_data.segment_identifies',
 "         WHERE dt >= '2018-07-01'",
 '           AND anonymous_id IS NOT NULL',
 '           AND user_id IS NOT NULL ) a',
 '      LEFT JOIN',
 '        (SELECT id,',
 '                email,',
 '                created',
 '         FROM mapbox_customer_data.accounts',
 "         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id",
 '      LEFT JOIN',
 '        (SELECT anonymous_id AS anon_id_ad,',
 '                context_campaign_name,',
 '                min(TIMESTAMP) AS min_exposure',
 '         FROM mapbox_customer_data.segment_pages',
 "         WHERE dt >= '2018-07-01'",
 '           AND context_campaign_name IS NOT NULL',
 '         GROUP BY 1,',
 '           

In [15]:
has_subquery

True

In [19]:
# query = '\n'.join(query_list).lstrip('\n')
formatter = column_parser.Parser(query_3)
formatted = formatter.format_query(query_3)
cte_dict = formatter.parse_cte(formatted)

In [20]:
cte_dict.keys()

dict_keys(['opportunity_to_name', 'main'])

In [22]:
print(cte_dict

{'opportunity_to_name': "SELECT id AS account_id,\n          name AS account_name\n   FROM sfdc.accounts sfdc_accounts\n   WHERE dt = '{run_date}'\n   GROUP BY id,\n            name)\n", 'main': 'SELECT *\nFROM opportunity_to_name'}


In [18]:
print(cte_dict['non_reg_users'])

SELECT context_campaign_name,
          min_exposure,
          event,
          original_timestamp AS event_timestamp,
          context_traits_email AS event_email
   FROM
     (SELECT a.*,
             b.*
      FROM
        (SELECT anonymous_id AS anon_id_ad,
                context_campaign_name,
                min(original_timestamp) AS min_exposure
         FROM
           (SELECT context_campaign_name,
                   anonymous_id,
                   original_timestamp
            FROM mapbox_customer_data.segment_pages
            WHERE dt >= '2018-07-01'
              AND context_campaign_name IS NOT NULL )
         GROUP BY 1,
                  2) a
      LEFT JOIN
        (SELECT DISTINCT anonymous_id AS anon_id_event,
                         original_timestamp,
                         event,
                         context_traits_email
         FROM mapbox_customer_data.segment_tracks
         WHERE dt >= '2018-07-01'
           AND event LIKE 'submitted_%form'
    

In [20]:
print(cte_dict['reg_users'])

SELECT *
   FROM
     (SELECT a.*,
             b.*,
             c.*,
             d.*
      FROM
        (SELECT DISTINCT anonymous_id,
                         user_id
         FROM mapbox_customer_data.segment_identifies
         WHERE dt >= '2018-07-01'
           AND anonymous_id IS NOT NULL
           AND user_id IS NOT NULL ) a
      LEFT JOIN
        (SELECT id,
                email,
                created
         FROM mapbox_customer_data.accounts
         WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ) b ON a.user_id = b.id
      LEFT JOIN
        (SELECT anonymous_id AS anon_id_ad,
                context_campaign_name,
                min(TIMESTAMP) AS min_exposure
         FROM mapbox_customer_data.segment_pages
         WHERE dt >= '2018-07-01'
           AND context_campaign_name IS NOT NULL
         GROUP BY 1,
                  2) c ON a.anonymous_id = c.anon_id_ad
      LEFT JOIN
        (SELECT DISTINCT anonymous_id AS anon_id_event,
                  

In [21]:
print(cte_dict['mql_flag'])

SELECT email,
          created_date,
          last_mql_date_c,
          mql_flag
   FROM
     (SELECT email,
             min(created_date) created_date,
             max(last_mql_date_c) last_mql_date_c,
             CASE
                 WHEN max(last_mql_date_c) IS NOT NULL THEN 1
                 ELSE 0
             END AS mql_flag,
             sum(CASE
                     WHEN is_deleted = TRUE THEN 1
                     ELSE 0
                 END) AS is_deleted
      FROM sales.salesforce_leads
      WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY
      GROUP BY 1)
   WHERE mql_flag = 1
     AND is_deleted = 0 ),



In [22]:
print(cte_dict['cleaned_list'])

SELECT DISTINCT *
   FROM
     (SELECT context_campaign_name,
             min_exposure,
             'created_an_account' AS event,
             created AS event_timestamp,
             email AS event_email
      FROM reg_users
      WHERE to_unixtime(min_exposure) <= to_unixtime(created)
        AND cast(min_exposure AS DATE) >= cast(created AS DATE) - INTERVAL '28' DAY
      UNION ALL SELECT context_campaign_name,
                       min_exposure,
                       event,
                       original_timestamp AS event_timestamp,
                       context_traits_email AS event_email
      FROM reg_users
      WHERE to_unixtime(min_exposure) <= to_unixtime(original_timestamp)
        AND cast(min_exposure AS DATE) >= cast(original_timestamp AS DATE) - INTERVAL '28' DAY
      UNION ALL SELECT *
      FROM non_reg_users)) 


In [23]:
print(cte_dict['main'])

SELECT a.*,
                                  b.*
FROM cleaned_list a
LEFT JOIN mql_flag b ON a.event_email = b.email) custom_sql_query


In [ ]:
users_query = cte_dict['reg_users']

In [ ]:
users_query.lstrip('')

In [ ]:
formatter = column_parser.Parser(users_query)
formatted = formatter.format_query(users_query)
query_list = formatted.split('\n')
query_list, has_subquery = has_child(query_list)